In [5]:
from utils.image_processing import vectorize
import os
import numpy as np
from tqdm import tqdm

In [6]:
label_map = {
    '+': 10,
    '-': 11,
    'times': 12,
    'forward_slash': 13,
    '(': 14,
    ')': 15
}

X = []
y = []
file_names = []

for symbol in os.listdir("small_dataset/"):
    try:
        label = int(symbol)
    except:
        label = label_map[symbol]
    
    for image in tqdm(os.listdir(os.path.join('small_dataset', symbol))):
        image_path = os.path.join('small_dataset', symbol, image)
        features = vectorize(image_path)
        X.append(features)
        file_names.append(image_path)
        y.append(label)

X = np.array(X)
y = np.array(y)

100%|██████████| 199/199 [00:00<00:00, 9002.40it/s]


In [7]:
np.savez('small_data.npz', X=X, y=y)
with open('small_filepaths.txt', 'w') as out:
    out.write('\n'.join(file_names))